# Excel to/from SBML
Peter Schubert (2020-11-15)
<br> <br>

In [3]:
import os
import time
import sbmlxdf

In [ ]:
# Ecoli models

xlsx_dir = 'xlsx_models/ecoli'
sbml_dir = 'SBML_models/ecoli'

# model_name = 'iJO1366_ccFBA'
# model_name = 'iJO1366_MOMENT'
# model_name = 'iJO1366_MOMENTmr'
model_name = 'iJO1366_GECKO'


In [2]:
# Yeast models
xlsx_dir = 'xlsx_models/yeast'
sbml_dir = 'SBML_models/yeast'

model_name = 'ecYeastGEM_batch_8_3_4'

In [5]:
# Synechocystis models
xlsx_dir = 'xlsx_models/synechocystis'
sbml_dir = 'SBML_models/synechocystis'

model_name = 'iJN678_predicted_GECKO'

In [3]:
print(os.getcwd())

/Users/peter/Documents/CCB/work/GECKO


## convert SBML model to xlsx model

In [6]:
sbml_name = os.path.join(sbml_dir, model_name) + '.xml'
if os.path.exists(sbml_name):
    model = sbmlxdf.Model(sbml_name)
    print(sbml_name, 'imported')
    print("last modified: %s" % time.ctime(os.path.getmtime(sbml_name)))
else:
     print(f'ERROR, not found!!! {sbml_name}')

SBML_models/synechocystis/iJN678_predicted_GECKO.xml imported
last modified: Sun Oct 27 12:48:01 2024


In [7]:
model.validate_sbml()

{}

In [8]:
xlsx_name = os.path.join(xlsx_dir, model_name) + '.xlsx'
model.to_excel(xlsx_name)
print(xlsx_name, 'exported')

xlsx_models/synechocystis/iJN678_predicted_GECKO.xlsx exported


## convert Excel to SBML model

In [24]:
xlsx_name = os.path.join(xlsx_dir, model_name) + '.xlsx'
if os.path.exists(xlsx_name):
    model = sbmlxdf.Model(xlsx_name)
    print(xlsx_name, 'imported')
    print("last modified: %s" % time.ctime(os.path.getmtime(xlsx_name)))
else:
     print(f'ERROR, not found!!! {xlsx_name}')

xlsx_models/eciJO1366_ccfba.xlsx imported
last modified: Sat Jun 24 15:55:15 2023


In [25]:
model.validate_sbml()

{}

In [26]:
sbml_name = os.path.join(sbml_dir, model_name) + '.xml'
model.export_sbml(sbml_name)
print(sbml_name, 'exported')

SBML_models/eciJO1366_ccfba.xml exported


## convert reactants - products table to reactants string

In [15]:
import pandas as pd
import re

In [66]:
xlsx_name = os.path.join(xlsx_dir, model_name) + '.xlsx'
with pd.ExcelFile(xlsx_name) as xlsx:
    df_reactions = pd.read_excel(xlsx, sheet_name = 'reactions')

In [74]:
for idx, row in df_reactions.iterrows():
    new_srefs = []
    for sref in row['reactants'].split(';'):
        params = sbmlxdf.extract_params(sref)
        if 'stoic' not in params:
            params['stoic'] = '1.0'
        params['const'] = True
        new_srefs.append(', '.join([f'{key}={val}' for key, val in params.items()]))
    df_reactions.at[idx, 'new_reactants'] = '; '.join(new_srefs)  
    new_srefs = []
    for sref in row['products'].split(';'):
        params = sbmlxdf.extract_params(sref)
        if 'stoic' not in params:
            params['stoic'] = '1.0'
        params['const'] = True
        new_srefs.append(', '.join([f'{key}={val}' for key, val in params.items()]))
    df_reactions.at[idx, 'new_products'] = '; '.join(new_srefs)  

In [30]:
df_gps = pd.DataFrame(gp2label.values(), index=gp2label, columns=['label'])
df_gps.index.name='id'
df_gps.head()

,label
id,
G_Q0045,Q0045
G_Q0250,Q0250
G_Q0275,Q0275
G_YDL067C,YDL067C
G_YEL039C,YEL039C


In [79]:
xlsx_name = 'tmp.xlsx'
with pd.ExcelWriter('tmp.xlsx') as writer:  
    df_reactions.to_excel(writer, sheet_name='fbcGeneProducts')


In [21]:
import pandas as pd

In [18]:
m_dict = model.to_df()

In [19]:

m_dict['parameters'].iloc[0]

value                       0.0
units       mmol_per_gDW_per_hr
constant                   True
Name: fbc_0_bound, dtype: object

In [24]:
s_bound = pd.Series([0.1, 'mmol_per_gDW_per_hr', True], index=['value', 'units', 'constant'])
s_bound.name = 'fbc_r_1001_lb'
s_bound

value                       0.1
units       mmol_per_gDW_per_hr
constant                   True
Name: fbc_r_1001_lb, dtype: object